In [4]:
import face_recognition
from PIL import Image
import cv2 
import os

In [2]:
image = face_recognition.load_image_file("foto.JPG")
print(image.shape)
face_locations = face_recognition.face_locations(image)

(648, 1152, 3)


In [3]:
face_locations

[(204, 491, 247, 448),
 (257, 919, 308, 867),
 (233, 765, 276, 722),
 (198, 163, 260, 100),
 (218, 601, 254, 565),
 (223, 847, 266, 804),
 (205, 337, 257, 285),
 (170, 733, 206, 697),
 (250, 1057, 286, 1021)]

In [4]:
image = cv2.imread("foto.JPG")

In [6]:
for (top, right, bottom, left) in face_locations:
    cv2.rectangle(image, (left, top), (right, bottom), (0, 0, 255), 2)
 
cv2.imshow("image", image) 
  
cv2.waitKey(0) 
cv2.destroyAllWindows() 

In [2]:
capture = cv2.VideoCapture(0)
capture.set(3, 512)
capture.set(4, 512)

while (capture.isOpened()):
    ret, frame = capture.read()
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) 
    face_locations = face_recognition.face_locations(frame)
    print(face_locations)
    if ret == True:

        for (top, right, bottom, left) in face_locations:
            cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)
        if cv2.waitKey(25) in (ord('q'), ord('Q')):
            break
    else: 
        break
    
    cv2.imshow('Frame', frame) 

capture.release() 
cv2.destroyAllWindows()

[]
[]
[]
[]
[]
[]
[]
[]
[(150, 408, 305, 253)]
[(139, 428, 325, 242)]
[(142, 464, 365, 241)]
[(142, 489, 365, 266)]
[(142, 489, 365, 266)]
[(142, 489, 365, 266)]
[(142, 489, 365, 266)]
[(142, 489, 365, 266)]
[(142, 489, 365, 266)]
[(160, 489, 345, 304)]
[]
[(142, 489, 365, 266)]
[(139, 469, 325, 283)]


Attandance

In [3]:
image = cv2.imread(r".\Resources\background.png")
cv2.imshow("image", image) 
  
cv2.waitKey(0) 
cv2.destroyAllWindows() 

In [66]:
import pickle
import numpy as np
import cvzone
from firebase_admin import db
import firebase_admin
from firebase_admin import credentials
from firebase_admin import storage
from datetime import datetime

In [40]:
if not firebase_admin._apps:
    cred = credentials.Certificate("serviceAccountKey.json")
    firebase_admin.initialize_app(cred,{
        'databaseURL': 'https://attandance-66c54-default-rtdb.asia-southeast1.firebasedatabase.app/',
        'storageBucket': 'attandance-66c54.appspot.com'
})
bucket = storage.bucket()

# Perlu ditambahkan jika tidak terdeteksi muka

In [76]:
img_background =  cv2.imread(r".\Resources\background.png")
img_background = cv2.resize(img_background, (1280, 512), fx=1, fy=1)

folder_mode_path = r".\Resources\Modes"
mode_path_list = os.listdir(folder_mode_path)
img_mode_list = []

print("loading encoded file")
file_encode = open(r".\model\Encode\my_EncodeFile.p", "rb")
encodeListKnown = pickle.load(file_encode)
file_encode.close()
encoded_img_list, id_list = encodeListKnown

print("encoded file collected")

modeType = 0
counter = 0

# get images mode and resize them
for path in mode_path_list:
    img_mode = cv2.imread(os.path.join(folder_mode_path, path))
    img_mode = cv2.resize(img_mode, (430, 480), fx=1, fy=1)
    img_mode_list.append(img_mode)

cap = cv2.VideoCapture(0)
cap.set(3, 128)
cap.set(4, 128)

while True:
    succes, img = cap.read()
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    face_locations = face_recognition.face_locations(img_rgb)
    encode_frames = face_recognition.face_encodings(img_rgb, face_locations)
    
    for encode_frame, face_loc in zip(encode_frames, face_locations):
        # check the matches face encoded
        matches = face_recognition.compare_faces(encoded_img_list, encode_frame)
        # magnitude of the distance similarity
        face_distance = face_recognition.face_distance(encoded_img_list, encode_frame)
        
        best_match_index = np.argmin(face_distance)
        if matches[best_match_index]:
            person_id = id_list[best_match_index]
            
            # cvzone.putTextRect(
            #         img, f"Face Detected, person id : {person_id[0]}", (20, 100),
            #         scale=1, thickness=1, 
            #         colorT=(0, 0, 255), colorR=(255, 100, 0), 
            #         font=cv2.FONT_HERSHEY_PLAIN, 
            #         offset=1,  
            #         border=1, colorB=(0, 5, 0)
            #     )

        
            for top, right, bottom, left in face_locations:
                # Scale back up face locations since the frame we detected in was scaled to 1/4 size
                top -= 60
                right += 20
                bottom += 60
                left -= 20
                # Draw a box around the face
                cv2.rectangle(img, 
                              (left, top), 
                              (right, bottom), 
                              (0, 0, 255), 
                              2)

                # Draw a label with a name below the face
                cv2.rectangle(img, 
                              (left, bottom - 35), 
                              (right, bottom), 
                              (0, 0, 255), 
                              cv2.FILLED)
                
                font = cv2.FONT_HERSHEY_DUPLEX
                cv2.putText(img, f" {person_id[0]}: {person_id[1]}", (left + 2, bottom - 2), font, 1.0, (255, 255, 255), 1)
            
            id = id_list[best_match_index]
            if counter == 0:
                counter = 1
                modeType = 1

        img_background[100:100+img.shape[0], 50:50+img.shape[1], :] = img
        img_background[20:20+img_mode_list[modeType].shape[0], 800:800+img_mode_list[modeType].shape[1], :] = img_mode_list[modeType]    
        
        if counter != 0:
            if counter == 1:
                person_info = db.reference(f"people/{id[0]}").get()
                blob = bucket.get_blob(f"images/{id[0]}_{id[1]}.JPG")
                array = np.frombuffer(blob.download_as_string(), np.uint8)
                img_person = cv2.imdecode(array, cv2.COLOR_BGRA2BGR)
                img_person = cv2.resize(img_person, (230, 240), fx=1, fy=1)

                datetimeObject = datetime.strptime(person_info['last_attandance'],"%Y-%m-%d %H:%M:%S")
                secondsElapsed = (datetime.now()-datetimeObject).total_seconds()
                if secondsElapsed > 60:
                    ref = db.reference(f"people/{id[0]}")
                    person_info['total_attandance'] += 1
                    ref.child('total_attandance').set(person_info['total_attandance'])
                    ref.child('last_attandance').set(datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
                else:
                    modeType = 3
                    counter = 0
                    img_background[20:20+img_mode_list[modeType].shape[0], 800:800+img_mode_list[modeType].shape[1], :] = img_mode_list[modeType]    
                    
            if modeType != 3:
                
                if 10<counter<20:
                    modeType = 2
                    
                img_background[20:20+img_mode_list[modeType].shape[0], 800:800+img_mode_list[modeType].shape[1], :] = img_mode_list[modeType]    
                    
                if counter <=10:
                    cv2.putText(img_background, 
                                str(person_info['total_attandance']), 
                                (840, 80),
                                cv2.FONT_HERSHEY_COMPLEX, 
                                0.5, 
                                (100, 100, 100), 
                                1)
                    cv2.putText(img_background, 
                                str(id[0]), 
                                (1000, 360), 
                                cv2.FONT_HERSHEY_COMPLEX, 
                                0.5, 
                                (100, 100, 100), 
                                1)
                    cv2.putText(img_background, 
                                str(person_info['major']), 
                                (1000, 403), 
                                cv2.FONT_HERSHEY_COMPLEX, 
                                0.5, 
                                (100, 100, 100), 
                                1)
                    cv2.putText(img_background, 
                                str(person_info['starting_year']), 
                                (1110, 463), 
                                cv2.FONT_HERSHEY_COMPLEX, 
                                0.5, 
                                (100, 100, 100), 
                                1)
                    
                    (w, h), _ = cv2.getTextSize(person_info['name'], cv2.FONT_HERSHEY_SIMPLEX, 1, 1)
                    offset = (430 - w) // 2
                    cv2.putText(img_background, 
                                str(person_info['name']), 
                                (840 + offset, 320), 
                                cv2.FONT_HERSHEY_COMPLEX, 
                                0.5, 
                                (100, 100, 100), 
                                1)
                    
                    img_background[50:50+img_person.shape[0], 900:900+img_person.shape[1]]= img_person
                
                counter += 1
                
                if counter >=20:
                    counter = 0
                    modeType = 0
                    person_info = []
                    img_person = []
                    img_background[20:20+img_mode_list[modeType].shape[0], 800:800+img_mode_list[modeType].shape[1], :] = img_mode_list[modeType]  
            
    cv2.imshow("image Background", img_background)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release() 
cv2.destroyAllWindows()
    

loading encoded file
encoded file collected
